Get data

In [ ]:
!wget https://toyokeizai.net/sp/visual/tko/covid19/csv/data.csv

In [ ]:
import pandas as pd
from datetime import datetime
import altair as alt

In [ ]:
data = pd.read_csv('data.csv', parse_dates=['確定日'])

In [ ]:
# Remove 3 cases with 'under investigation' instead of date
data = data[data['確定日']!='調査中']

In [ ]:
# parse date
data['date'] = data['確定日'].apply(lambda x: datetime.strptime(f'2020/{x}', '%Y/%m/%d'))

In [ ]:
data.head()

In [ ]:
# regions translations
locations_jp = ['北海道', '愛知県', '東京都', '神奈川県', '千葉県', '和歌山県', '中国（武漢市）', '大阪府',
       '熊本県', '福岡県', '石川県', '沖縄県', '高知県', '京都府', '調査中', '兵庫県', '中国（湖北省）',
       '和歌山', '中国（湖南省）', '岐阜県', '埼玉県', '奈良県', '宮城県', '静岡県', '新潟県', '栃木県',
       '滋賀県', '神奈川', '長野県', '三重県']
locations = ['Hokkaido', 'Aichi', 'Tokyo', 'Kanagawa', 'Chiba', 'Wakayama', 'China (Wuhan)', 'Osaka',
       'Kumamoto', 'Fukuoka', 'Ishikawa', 'Okinawa', 'Kochi', 'Kyoto', 'Under investigation', 'Hyogo', 'China (Hubei)',
       'Wakayama', 'China (Hunan)', 'Gifu', 'Saitama', 'Nara', 'Miyagi', 'Shizuoka', 'Niigata', 'Tochigi',
       'Shiga Prefecture', 'Kanagawa', 'Nagano Prefecture', 'Mie Prefecture']
translations = dict(zip(locations_jp, locations))
data['Place of residence'] = data['居住地'].apply(lambda x: translations[x])

In [ ]:
# Sort the area per total number of case
sorted_areas= data.groupby('Place of residence').count().date.sort_values(ascending=False).index.values
sorted_areas

Chart definitions

In [ ]:
height=600

In [ ]:
Y_axis = alt.Axis(labels=True, domain=False, ticks=False, grid=False)
X_axis = alt.Axis(labels=True, domain=False, ticks=False, grid=False, orient='top')
totals_day = alt.Chart(data, width=50, height=height).transform_aggregate(
    [alt.AggregatedFieldDef(op='count', **{'as': 'date_total'})], groupby=['date']
).mark_rect().encode(
    alt.Y('monthdate(date)', axis=alt.Axis(labels=False, domain=False, ticks=False, grid=False), sort='descending', title=None),
    alt.X('date_total:Q', title='total'), 
    color=alt.value('gray')
)

In [ ]:
rect = alt.Chart(data, height=height).mark_rect().encode(
    alt.X('Place of residence:O', sort=list(sorted_areas), axis=X_axis),
    alt.Y('monthdate(date)', title='date', axis=Y_axis, sort='descending'), 
    alt.Color('count()', scale=alt.Scale(scheme='lightmulti'), title='New cases'),
)

In [ ]:
totals_area = alt.Chart(data, height=50).transform_aggregate(
    [alt.AggregatedFieldDef(op='count', **{'as': 'area_total'})], groupby=['Place of residence']
).mark_rect().encode(
    alt.X('Place of residence:O', sort=list(sorted_areas), axis=alt.Axis(title=None, labels=False, domain=False, ticks=False, grid=False)),
    alt.Y('area_total:Q', title='total'),
    color=alt.value('gray')
)

In [ ]:
viz1 = totals_area & (rect | totals_day)
viz1